# Solving Sudoku

The primary description of this coursework is available on the CM20252 Moodle page. This is the Jupyter notebook you must complete and submit to receive marks. 

** You must follow all instructions given in this notebook. **

Restart the kernel and run all cells before submitting the notebook. This will guarantee that we will be able to run your code for testing.

Remember to save your work regularly. 

## Getting Started

You already know that you will be writing a Sudoku solver. You need to implement your solver in Python in this notebook. You can use any of the appropriate problem-solving techniques discussed in the lectures. You are encouraged to modify the basic algorithms. Be creative. 

You will be given Sudoku puzzles that either have a single solution or no solution. You will need to identify the solution, if there is one.

Below is a sample puzzle along with its solution. 

<img src="resources/images/Sudoku_unsolved.png" style="width: 200px;"/>
<img src="resources/images/Sudoku_solved.png" style="width: 200px;"/>


## Sample Sudokus

You can test your code on a set of 100 sample Sudoku puzzles. This set is similar to the test set that will be used to assess your work. 

The following code will load 100 Sudoku puzzles and their solutions into two `100x9x9` numpy arrays. Empty cells are designated by zeros. 

In [18]:
import numpy as np

# Load sudokus
sudokus = np.load("resources/data/sudokus.npy")
print("Shape of sudokus array:", sudokus.shape, "; Type of array values:", sudokus.dtype)

# Load solutions
solutions = np.load("resources/data/solutions.npy")
print("Shape of solutions array:", solutions.shape, "; Type of array values:", solutions.dtype, "\n")

# Print the first sudoku...
print("Sudoku #1:")
print(sudokus[0], "\n")

# ...and its solution
print("Solution of Sudoku #1:")
print(solutions[0])

Shape of sudokus array: (100, 9, 9) ; Type of array values: int32
Shape of solutions array: (100, 9, 9) ; Type of array values: int32 

Sudoku #1:
[[0 0 4 3 0 0 2 0 9]
 [0 0 5 0 0 9 0 0 1]
 [0 7 0 0 6 0 0 4 3]
 [0 0 6 0 0 2 0 8 7]
 [1 9 0 0 0 7 4 0 0]
 [0 5 0 0 8 3 0 0 0]
 [6 0 0 0 0 0 1 0 5]
 [0 0 3 5 0 8 6 9 0]
 [0 4 2 9 1 0 3 0 0]] 

Solution of Sudoku #1:
[[8 6 4 3 7 1 2 5 9]
 [3 2 5 8 4 9 7 6 1]
 [9 7 1 2 6 5 8 4 3]
 [4 3 6 1 9 2 5 8 7]
 [1 9 8 6 5 7 4 3 2]
 [2 5 7 4 8 3 9 1 6]
 [6 8 9 7 3 4 1 2 5]
 [7 1 3 5 2 8 6 9 4]
 [5 4 2 9 1 6 3 7 8]]


## Your code ##

Define a function called `sudoku_solver()` that takes **one** Sudoku puzzle (a $9 \times 9$ numpy array) as input and returns the solved Sudoku as a $9 \times 9$ numpy array. Note that the test set may contain invalid Sudokus, that is, Sudokus with no solution. In such a case, your function should return a $9 \times 9$ numpy array whose values are all equal to `-1`.  

You may use more than one cell to write your code (but this is not necessary).

In [19]:
import time


class Sudoku:

    def __init__(self, sudoku_grid):
        self.sudoku = sudoku_grid
        self.clone = np.copy(self.sudoku)  # Variable used in Backtracking

    def get_grid(self, cell):
        # gets the grid of cell
        grid = np.zeros((3, 3))
        col = ((cell[1] // 3) * 3)
        row = ((cell[0] // 3) * 3)
        n = 0
        m = 0

        for i in range(row, row + 3):
            for j in range(col, col + 3):
                grid[n][m] = self.sudoku[i][j]
                m += 1
            n += 1
            m = 0

        return grid

    def get_empty_cells(self):
        # returns list of cells which are empty
        cells = []
        for row in range(0, 9):
            for col in range(0, 9):
                if self.sudoku[row, col] == 0:
                    cells.append((row, col))
        return cells

    def get_non_empty_cells(self):
        # returns cells which have been assigned a digit
        cells = []
        for row in range(0, 9):
            for col in range(0, 9):
                if self.sudoku[row, col] > 0:
                    cells.append((row, col))
        return cells

    def get_valid_digits(self, cell):
        # Gets valid digits of cells
        digits = []
        horizontal = self.sudoku[cell[0], :]
        vertical = self.sudoku[:, cell[1]]
        grid = self.get_grid(cell)

        for digit in range(1, 10):
            if digit not in vertical:
                if digit not in horizontal:
                    if digit not in grid:
                        digits.append(digit)
        return digits

    def constraint_search_complete(self, cells):

        # Methods checks to see if the no more constraint propogation can take place, 
        # if there are no immediate solutions to the cells it returns True, else False.

        if -1 in self.sudoku:
            return True
        else:
            digits = []
            for cell in cells:
                digit = self.get_valid_digits(cell)
                digits.append(digit)
            for digit in digits:
                if len(digit) == 1:
                    return False
            return True

    def is_unsolvable(self):
        # Changes digits in all cells to -1
        for i in range(0, 9):
            for j in range(0, 9):
                self.sudoku[i][j] = -1

    def check_sudoku(self):
        # Checks the state of current sudoku
        cells = self.get_non_empty_cells()
        for cell in cells:
            horizontal = self.sudoku[cell[0], :]
            vertical = self.sudoku[:, cell[1]]
            grid = self.get_grid(cell)
            digit = self.sudoku[cell[0]][cell[1]]
            if (np.count_nonzero(horizontal == digit) > 1) or (np.count_nonzero(vertical == digit) > 1) or (
                    np.count_nonzero(grid == digit) > 1):
                return False
        return True

    def check_domains(self):
        # Checks the domains of all cells
        cells = self.get_empty_cells()
        for cell in cells:
            digits = self.get_valid_digits(cell)
            for digit in digits:
                if not digit:
                    return False
        return True

    def is_complete(self):
        # Checks to see if sudoku is complete
        if -1 in self.sudoku:
            return False

        for i in range(0, 9):
            for j in range(0, 9):
                if self.sudoku[i][j] == 0:
                    return False
        if self.check_sudoku:
            return True
        else:
            return False

    def constraint_propagation(self):
        # Loop for finding cells with only one solution,
        # stops when there are none left.
        while not self.constraint_search_complete(self.get_empty_cells()):
            cells = self.get_empty_cells()
            for cell in cells:
                digits = self.get_valid_digits(cell)
                if len(digits) == 1:
                    self.sudoku[cell[0], cell[1]] = digits[0]

def backtracking(sudoku):
    # Backtracking search, tries each avaialble digit individually
    if sudoku.is_complete():
        return True
    cells = sudoku.get_empty_cells()
    for cell in cells:
        digits = sudoku.get_valid_digits(cell)
        for digit in digits:
            sudoku.sudoku[cell[0], cell[1]] = digit
            sudoku.clone = np.copy(sudoku.sudoku)
            sudoku.constraint_propagation() # Uses constraint solving in tandem, to try improve effcieny
            if sudoku.is_complete():
                return True
            else:
                sudoku.sudoku = np.copy(sudoku.clone)
            if backtracking(sudoku):
                return True
            else:
                sudoku.sudoku[cell[0], cell[1]] = 0 # If digit fails, assign back to zero. 
        return False

def sudoku_solver(unsolved_sudoku):

    # Creates sudoku object
    s = Sudoku(unsolved_sudoku)

    # Checks sudoku to make sure given sudoku is valid.
    if not s.check_sudoku():
        s.is_unsolvable()
        return s.sudoku
    else:
        # Uses constraint propagation to start, to check if any cells with one solution can be found.
        s.constraint_propagation()
        if s.is_complete():
            return s.sudoku
        if s.check_domains():
            if backtracking(s):
                return s.sudoku
            else:
                s.is_unsolvable()
                return s.sudoku
        else:
            s.is_unsolvable()
            return s.sudoku

## Testing your code

You can test your code on the sudoku puzzles that we have provided. For example, the following code cell should return `True` if your `sudoku_solver()` function can correctly solve the first sudoku. This will work only if all of your code is above the test cell. Otherwise, the test cell does not have access to the `sudoku_solver()` function.

In [21]:
import time

t = time.process_time()
n = 0
for i in range(0, 100):
    if np.array_equal(sudoku_solver(sudokus[i]), solutions[i]):
        n += 1
print("Solved: ", n)
elapsed_time = time.process_time() - t
print(elapsed_time)

Solved:  100
0.31892299999999807


### How we will test your code

We will test your code using the hidden tests in the following cell. Specifically, the hidden tests will test your `sudoku_solver()` function on a different set of 100 sudoku puzzles of similar difficulty. **Make sure all of your code is above our test cell. ** Otherwise, the test cell will not have access to the sudoku_solver() function and you will receive zero marks.

In [ ]:
# This is the TEST CELL. Do not delete or change. All of your code must be written above this cell. 